## Importação das BIblitoecas

In [1]:
import pandas as pd
import datetime
import matplotlib
import openpyxl
import dataframe_image as dfi

<h1><b style='color:orange'> 1° Análise </b> - Identificar possíveis clientes  </h1>

<p>
    Qual o perfil de clientes que desejamos atingir:
</p>
<p>
    <b> Resposta: </b> Clientes que estão insatisfeitos com a Russia em relação a Guerra
</p>

### Apresentar a distribuição dos paises em relação a rússia

In [3]:
# Leitura dos dados
df_relacao_russia = pd.read_excel('Thanael - Data Glow Up 33.xlsx', sheet_name='support_russian')

# Remover colunas desnecessárias
df_relacao_russia.drop(columns = df_relacao_russia.columns[2:] , inplace = True)

# Agrupar os dados de acordo com a resposta do país
df_analise_relacao = df_relacao_russia.groupby('response to russia',as_index = False).count()

# Criar a coluna de Percentual
df_analise_relacao['%'] =  df_analise_relacao['country'].apply(lambda quantidade: round(quantidade / df_analise_relacao['country'].sum(),2)*100)

# Ordernar os dados
df_analise_relacao = df_analise_relacao.sort_values(by = 'country', ascending=False)

# Renomear as colunas
df_analise_relacao.columns = ['response','quantity','%']

# Resetar o Index para o print haha
df_analise_relacao = df_analise_relacao.reset_index(drop = True)

In [4]:
df_analise_relacao_style = (
df_analise_relacao.style
    .format('{:.0f}%', na_rep='MISS',subset=['%'])
    .background_gradient(cmap='YlGn',subset=['%','quantity'])
    .applymap(lambda v: 'font-weight: bold', subset=['response'])
)

C:\Users\thanael.butewicz\AppData\Local\Temp\ipykernel_19608\956880900.py:5: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(lambda v: 'font-weight: bold', subset=['response'])


In [5]:
df_analise_relacao_style

,response,quantity,%
0,UNHAPPY,49,35%
1,CONDEMNS RUSSIA,42,30%
2,NEUTRAL,22,16%
3,LEANS RUSSIA,19,13%
4,PRO RUSSIA,9,6%


In [6]:
dfi.export(df_analise_relacao_style, 'relacao_russia.png')

### Identificar os países que estão Insatisfeitos com a Rússia

In [7]:
#Encontrar os paises que estão insatisfeitos com a Russia (UNHAPPY e CONDEMNS RUSSIA) 
possiveis_clientes = df_relacao_russia[df_relacao_russia['response to russia'].isin(['UNHAPPY','CONDEMNS RUSSIA'])]
print('\n\n Paises Insatisfeitos com a Rússia: \n')
possiveis_clientes



 Paises Insatisfeitos com a Rússia: 



,country,response to russia
4,Argentina,UNHAPPY
6,Australia,CONDEMNS RUSSIA
7,Austria,CONDEMNS RUSSIA
12,Belgium,CONDEMNS RUSSIA
13,Belize,UNHAPPY
...,...,...
159,United Kingdom,CONDEMNS RUSSIA
160,United States,CONDEMNS RUSSIA
161,Uruguay,UNHAPPY
165,Wales,CONDEMNS RUSSIA


<h1><b style='color:orange'> 2° Análise </b> - Identificar países que importam Petróleo da Rússia  </h1>

#### Função para adicionar mascara de valor em Número

In [8]:
def formatar_valor_monetario(valor):
    sufixos = {
        1e3: 'K',
        1e6: 'M',
        1e9: 'B',
        1e12: 'T',
        1e15: 'Q'
    }
    
    for sufixo, simbolo in sorted(sufixos.items(), reverse=True):
        if valor >= sufixo:
            valor_formatado = "{:.1f} {}".format(valor / sufixo, simbolo)
            return valor_formatado

    # Se o valor for menor que 1 mil, retorne-o sem modificação
    return str(valor)

### Países importam Petróleo da Rússia

In [9]:
# Leitura dos dados
df_import_oil = pd.read_excel('Thanael - Data Glow Up 33.xlsx', sheet_name='import_russian_oil')

# Ajustar a 1° Linha para ser as colunas
df_import_oil.columns = df_import_oil.iloc[0]

# Remover as colunas que não vão ser usadas
df_import_oil.drop(columns = df_import_oil.columns[3:] , inplace = True)

# Remover a primeira linha
df_import_oil = df_import_oil.iloc[1:]

### Explicação dos dados

- **Name:** País
- **Share %:** Percentual do total de Petróleo russo exportado
- **Gross Export:** $

In [42]:
df_import_oil.head(10)

,Name,Share %,Gross Export
1,Australia,0.090141,103537184
2,Austria,0.102661,117916848
3,Azerbaijan,0.108618,124759304
4,Belarus,5.090166,5846611456
5,Belgium,0.086559,99422200
6,Bulgaria,1.388623,1594985216
7,Canada,0.291549,334876000
8,China,30.823524,35404181504
9,Colombia,0.020745,23828054
10,Croatia,0.400097,459554848


### Países descontentes com a Rússia, mas que importam Petróleo Russo

In [11]:
# Fazer o JOIN dos dados
df_potencial = pd.merge(
    left = df_import_oil,
    right = possiveis_clientes,
    left_on = 'Name',
    right_on = 'country',
    how = 'inner'
)

# Ordenar pela % de pétroleo da Rússia que o país importa
df_potencial = df_potencial.sort_values(by='Share %', ascending = False)

# Criar uma coluna com mascara de valores
df_potencial['Gross Export - Formatado'] = df_potencial['Gross Export'].apply(lambda valor: formatar_valor_monetario(valor))

# Filtrar as colunas
df_potencial = df_potencial[['country','response to russia','Share %','Gross Export','Gross Export - Formatado', ]]

# Resetar o Index
df_potencial = df_potencial.reset_index(drop = True)

In [16]:
# Fazer o Style para colocar no README

df_potencial_style = (
df_potencial.head(20).style
    .format('{:.0f}%', na_rep='MISS',subset=['Share %'])
    .background_gradient(cmap='YlGn',subset=['Share %','Gross Export'])
    .applymap(lambda v: 'font-weight: bold', subset=['country','Share %','Gross Export - Formatado'])
)

C:\Users\thanael.butewicz\AppData\Local\Temp\ipykernel_19608\137552543.py:7: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(lambda v: 'font-weight: bold', subset=['country','Share %','Gross Export - Formatado'])


In [18]:
dfi.export(df_potencial_style, 'potenciais_clientes.png')

In [19]:
df_potencial_style

,country,response to russia,Share %,Gross Export,Gross Export - Formatado
0,Germany,CONDEMNS RUSSIA,10%,11448549376,11.4 B
1,Netherlands,CONDEMNS RUSSIA,10%,11149848576,11.1 B
2,Poland,CONDEMNS RUSSIA,6%,6572766208,6.6 B
3,South Korea,CONDEMNS RUSSIA,5%,5466311680,5.5 B
4,Finland,CONDEMNS RUSSIA,4%,4521537536,4.5 B
5,Italy,CONDEMNS RUSSIA,3%,3740185344,3.7 B
6,Japan,CONDEMNS RUSSIA,3%,3164011008,3.2 B
7,Turkey,UNHAPPY,3%,3152163584,3.2 B
8,France,CONDEMNS RUSSIA,2%,2213134592,2.2 B
9,Sweden,CONDEMNS RUSSIA,2%,1756946304,1.8 B
